<a href="https://colab.research.google.com/github/Martine1grace/NLP_FELLOWSHIP_AI-Machine_Translation.-/blob/main/Week4/EasyNMT_Colab_REST_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EasyNMT REST API using Colab

This notebook demonstrates, how we can create a REST API on Google Colab that runs [EasyNMT](https://github.com/UKPLab/EasyNMT).

You can then query this API from your local machine using standard GET / POST requests and translate documents to 150+ languages.

This allows you to use the free GPU from Colab for Neural Machine Translation with the simplicity of quering a REST API for machine translation.

**Note**: **Colab has a runtime restrictions**, that a notebook can only run up to a certain time limit. It is then stopped after this time limit. This notebook will **not work** if you want to host a translation API permanently. But you can use this notebook to start a REST API, and then calling it from your machine to translate larger amount of text. [more info](https://research.google.com/colaboratory/faq.html#resource-limits)



## Colab with GPU
When running this notebook in colab, ensure that you run it with a GPU as hardware accelerator. To enable this:
- Navigate to Edit → Notebook Settings
- select GPU from the Hardware Accelerator drop-down

With `!nvidia-smi` we can check which GPU was assigned to us in Colab.

In [ ]:
!nvidia-smi

Tue Mar 16 21:41:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## EasyNMT Installation
First we install EasyNMT

In [ ]:
!pip install easynmt

## Other dependencies
We create our REST API using [FastAPI](https://fastapi.tiangolo.com/). In order to host this in a Colab, we use [pyngrok](https://github.com/alexdlaird/pyngrok).


In [ ]:
!pip install fastapi pyngrok  uvicorn nest-asyncio 

## Translation Model
Then we define our translation model. See the [EasyNMT documentation](https://github.com/UKPLab/EasyNMT) for more details

In [ ]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

# REST API
Next we define a simple REST API.

We define a GET and a POST method, so that you can later query the REST API with GET and POST requests.

The API accepts three parameters:
- target_lang: Our target language for the translation
- text: A list of texts we want to translate
- source_lang: Optional. The source language for all texts

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from typing import Optional, Union, List
from fastapi import FastAPI, HTTPException, Query, Request

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
def translate_get(target_lang: str, text: List[str] = Query([]), source_lang: Optional[str] = None):
  return model.translate(text, target_lang=target_lang, source_lang=source_lang)

@app.post("/")
async def translate_post(request: Request):
    data = await request.json()
    return translate_get(**data)


# Host REST-API on Colab
Finally we start our REST API and host it on Colab.

As output, the public URL will be displayed which you can use to translate documents.

Either by using a GET request (e.g. opening the following URL in your browser):
`http://[auto_id_from_ngrok].ngrok.io?target_lang=en&text=[Your_Text]`

Or via a POST request:
```
import requests
r = requests.post("http://[auto_id_from_ngrok].ngrok.io", 
     json={'target_lang': 'en', 'text': ["Hallo Welt", "Es werden alle Texte übersetzt"]})
print(r.json())
```

Execute the following cell and let it run as long as you like to have the translation REST API hosted. Stop the next cell to shutdown the REST API.

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print("REST API started")
print("Your public API URL:", ngrok_tunnel.public_url)
print("You can for example open the following URL in your browser: {}?target_lang=en&text=Hallo%20Welt".format(ngrok_tunnel.public_url))

nest_asyncio.apply()
uvicorn.run(app, port=8000)



REST API started
Your public API URL: http://f96dc72838ff.ngrok.io
You can for example open the following URL in your browser: http://f96dc72838ff.ngrok.io?target_lang=en&text=Hallo%20Welt


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2003:d4:bf3f:113a:2455:cce5:914b:2876:0 - "GET /?target_lang=en&text=Hallo%20Welt HTTP/1.1" 200 OK


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


INFO:     2003:d4:bf3f:113a:2455:cce5:914b:2876:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     18.190.26.136:0 - "GET / HTTP/1.1" 422 Unprocessable Entity
INFO:     18.190.26.136:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     18.221.194.75:0 - "GET / HTTP/1.1" 422 Unprocessable Entity
INFO:     18.221.194.75:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
